In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/trec-2025/2025-starter-kit-main/.example.env
/kaggle/input/trec-2025/2025-starter-kit-main/main.py
/kaggle/input/trec-2025/2025-starter-kit-main/README.md
/kaggle/input/trec-2025/2025-starter-kit-main/produce_run.py
/kaggle/input/trec-2025/2025-starter-kit-main/output/tracking_data.json
/kaggle/input/trec-2025/2025-starter-kit-main/output/dragun-organizers-starter-kit-task-1
/kaggle/input/trec-2025/2025-starter-kit-main/output/dragun-organizers-starter-kit-task-2
/kaggle/input/trec-2025/2025-starter-kit-main/data/trec-2025-dragun-topics.jsonl
/kaggle/input/trec-2025/2025-starter-kit-main/modules/report_generator.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/segment_retriever.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/question_generator.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/query_generator.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/information_evaluator.py


In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth
    !pip install langchain langchain-community
    !pip install jsonlines
    !pip install --quiet langchain transformers sentencepiece

import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import json
from unsloth import FastLanguageModel
from unsloth import FastModel
import torch
from tqdm import tqdm
from unsloth.chat_templates import get_chat_template

2025-08-08 11:42:42.191516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754653362.367966      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754653362.425006      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
TOPICS_PATH = '/kaggle/input/trec-2025/2025-starter-kit-main/data/trec-2025-dragun-topics.jsonl'

def load_jsonl(fname):
    with open(fname, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]
topics = load_jsonl(TOPICS_PATH)
print("Loaded", len(topics), "topics.")

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)  

from langchain.prompts import PromptTemplate

Loaded 30 topics.
==((====))==  Unsloth 2025.8.1: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [4]:
prompt_template = PromptTemplate(
    input_variables=["title", "body"],
    template=(
        """You are a professional fact-checker and investigative media literacy expert, trained on resources like PolitiFact, MBFC, NewsQA, and FEVER. 
Your task: Generate 10 sharp, critical, and investigative questions a thoughtful reader should ask to assess the trustworthiness of a given news article. 
These must be specific to the article, ranked from most to least important for evaluating credibility.

Core focus areas for question generation:
1. Investigate the Source:
   - Publisher's or quoted sources' background, reputation, and biases.
   - Ownership, funding, and editorial policies.
   - Credentials and past track record.

2. Examine Claims and Evidence:
   - Central factual assertions and their reliability.
   - Quality of supporting evidence or missing context.
   - Alternative explanations not mentioned.

3. Trace Information Origins:
   - Original sources for quotes, data, or statistics.
   - Methodology of cited research.
   - How information flows from primary to secondary sources.

4. Assess Perspective and Balance:
   - Missing viewpoints or underrepresented perspectives.
   - Diversity of expert opinions.

5. Evaluate Context and Timing:
   - Broader events surrounding the story.
   - Potential motivations for timing.
   - Related developments that might shift interpretation.

Guidelines for each question:
- Max 300 characters.
- One question per idea (no compound questions).
- Avoid vague/general phrasing.
- Avoid questions fully answerable from the article itself; require external verification or deeper inquiry.
- Be actionable for a reader to investigate.

---

Example 1 (PolitiFact-style):
Title: "WHO: Processed meat causes cancer"
Article: WHO classified processed meat as carcinogenic...
Questions:
1. What peer-reviewed studies support WHO's claim?
2. Is the research methodology disclosed in the article?
3. Who funded the studies mentioned?
4. Are dissenting expert views included?
5. Are the health risks exaggerated?
6. Is primary data linked or cited?
7. Does the article mention affected demographics?
8. Are any advocacy groups quoted?
9. Is causal language justified?
10. Are the claims corroborated by other agencies?

---

Example 2 (MBFC-style):
Title: "CDC says masks are optional outdoors"
Article: CDC updates its outdoor masking guidance...
Questions:
1. Is there scientific consensus supporting this change?
2. Does the article quote data or just opinion?
3. Are counterarguments or dissenting experts cited?
4. Are political motives or timing relevant?
5. Is this consistent with international guidance?
6. Does the article omit relevant prior CDC guidance?
7. Are potential risks understated?
8. Does the article rely on official or third-party data?
9. Are specific regional risks acknowledged?
10. Is the framing objective or emotionally loaded?

---

Now, using the same investigative approach, write 10 questions for this article:

Title: {title}
Article text:
{body}

Output only the questions as a numbered list from 1 to 10, each ending with a question mark.
"""
    )
)

In [5]:
import re

# 2. Load LLM
pipe = pipeline(
    "text-generation",
    model=model,  # assume already loaded
    tokenizer=tokenizer,  # assume already loaded
    max_new_tokens=600,
    temperature=0.6,
    top_p=0.9,
    do_sample=True,
    return_full_text=False
)
llm = HuggingFacePipeline(pipeline=pipe)
llm_chain = LLMChain(prompt=prompt_template, llm=llm, output_key="text")

# 3. Regex extraction
pattern = re.compile(r"^\s*\d+[.)]\s*(.+)$")

def extract_unique_questions(raw_text):
    seen = set()
    questions = []
    for line in raw_text.strip().split("\n"):
        match = pattern.match(line.strip())
        if match:
            q = match.group(1).strip()
            if 0 < len(q) <= 300 and q not in seen:
                seen.add(q)
                questions.append(q)
    return questions

# 4. Inference Loop
results = []
for topic in topics:
    docid = topic['docid']
    title = topic.get('title', '').strip()
    body = topic.get('body', '').strip()[:2000]

    if not title and not body:
        print(f"⚠️ Skipping empty topic {docid}")
        continue

    questions = []
    for attempt in range(3):
        output = llm_chain.invoke({"title": title, "body": body})
        text = output['text'] if isinstance(output, dict) else output
        questions = extract_unique_questions(text)
        if len(questions) >= 10:
            questions = questions[:10]
            break
        print(f"🔁 Retry {attempt+1} for {docid}: Got {len(questions)}")

    # Pad with 'N/A' if fewer than 10
    while len(questions) < 10:
        questions.append("N/A")

    results.append({
        "docid": docid,
        "questions": questions
    })

# 5. Print Example Output
for item in results[:3]:
    print(f"\nDocID: {item['docid']}")
    for i, q in enumerate(item['questions'], 1):
        print(f"{i}. {q}")

/tmp/ipykernel_36/701901908.py:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
/tmp/ipykernel_36/701901908.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt_template, llm=llm, output_key="text")


🔁 Retry 1 for msmarco_v2.1_doc_21_861891150: Got 6
🔁 Retry 2 for msmarco_v2.1_doc_21_861891150: Got 2
🔁 Retry 3 for msmarco_v2.1_doc_21_861891150: Got 9
🔁 Retry 1 for msmarco_v2.1_doc_25_501708725: Got 6

DocID: msmarco_v2.1_doc_04_420132660
1. What specific historical records or documents confirm Sam Panopoulos as the inventor of the Hawaiian pizza?
2. Are there any credible sources or contemporaneous accounts from the 1960s that corroborate the claim of the pizza's creation in London, Ontario?
3. How does the article address the possibility that other chefs or restaurants may have independently created a similar dish around the same time?
4. What evidence supports the assertion that the Hawaiian pizza originated in Ontario rather than Hawaii, and how was this conclusion reached?
5. What is the methodology used to determine that the pizza was created in 1962, and are there any primary sources from that year?
6. What is the role of the Naples stop during Sam Panopoulos's journey in inf

In [6]:
import pandas as pd

team_id = "CUET"
run_id = "CUET-qwen14B-v5"

submission_rows = []

for item in results:
    topic_id = item["docid"]
    for rank, question in enumerate(item["questions"], 1):
        # Clean question of unwanted tabs/newlines
        clean_q = question.replace('\t', ' ').replace('\n', ' ').strip()
        submission_rows.append([
            topic_id,
            team_id,
            run_id,
            rank,
            clean_q
        ])

# Create DataFrame and save
df = pd.DataFrame(submission_rows, columns=["topic_id", "team_id", "run_id", "rank", "question"])
df.to_csv("/kaggle/working/CUET_run10.tsv", sep="\t", index=False, header=False, encoding="utf-8")

In [7]:
# Show duplicated questions (exact text matches)
duplicates = df[df.duplicated('question', keep=False)]
num_duplicates = len(duplicates)

print(f"🔍 Found {num_duplicates} duplicated question rows.\n")
if num_duplicates > 0:
    print("📄 Duplicated questions (full rows):")
    display(duplicates)

# Frequency count of duplicate questions
print("\n📊 Questions that appear more than once:")
duplicate_counts = df['question'].value_counts()
display(duplicate_counts[duplicate_counts > 1])

# Check for 'N/A' values (both literal string and NaN)
na_mask = df['question'].isna() | (df['question'].str.strip().str.upper() == 'N/A')
na_rows = df[na_mask]

print(f"\n⚠️ Found {len(na_rows)} rows with 'N/A' or missing question values.\n")
if len(na_rows) > 0:
    print("🚫 Rows with N/A or missing questions:")
    display(na_rows)

🔍 Found 0 duplicated question rows.


📊 Questions that appear more than once:


Series([], Name: count, dtype: int64)


⚠️ Found 1 rows with 'N/A' or missing question values.

🚫 Rows with N/A or missing questions:


,topic_id,team_id,run_id,rank,question
49,msmarco_v2.1_doc_21_861891150,CUET,CUET-qwen14B-v5,10,N/A


In [8]:
# Step 2: Create dict of missing ranks per topic_id
missing_questions = {}
for _, row in na_rows.iterrows():
    topic_id = row["topic_id"]
    rank = int(row["rank"])
    if topic_id not in missing_questions:
        missing_questions[topic_id] = []
    missing_questions[topic_id].append(rank)

# Step 3: Load topics.jsonl into dict
topic_path = "/kaggle/input/trec-2025/2025-starter-kit-main/data/trec-2025-dragun-topics.jsonl"
topics_dict = {}
with open(topic_path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        docid = obj.get("docid", "").strip()
        topics_dict[docid] = {
            "title": obj.get("title", ""),
            "body": obj.get("body", "")
        }

# Step 4: Regenerate only missing ranks
patch_rows = []

for docid, ranks in missing_questions.items():
    if docid not in topics_dict:
        print(f"❌ Topic ID {docid} not found in topics.jsonl, skipping.")
        continue

    title = topics_dict[docid]["title"]
    body = topics_dict[docid]["body"][:2000]

    print(f"🔁 Regenerating for topic {docid}...")

    full_questions = []
    for attempt in range(3):
        output = llm_chain.invoke({"title": title, "body": body})
        text = output["text"] if isinstance(output, dict) else output
        full_questions = extract_unique_questions(text)
        if len(full_questions) >= 10:
            break
        print(f"  Retry {attempt + 1}: Only got {len(full_questions)}")

    # Pad to 10 if still short
    while len(full_questions) < 10:
        full_questions.append("N/A")

    # Add only the requested ranks
    for rank in ranks:
        patch_rows.append({
            "topic_id": docid,
            "team_id": "CUET",
            "run_id": "CUET-qwen14B-v5",
            "rank": rank,
            "question": full_questions[rank - 1]
        })

# Step 5: Patch into original DataFrame
patch_df = pd.DataFrame(patch_rows)

# Remove N/A rows
df = df[~na_mask]

# Combine and sort
df_final = pd.concat([df, patch_df], ignore_index=True).sort_values(by=["topic_id", "rank"])

# Step 6: Save without header
df_final.to_csv("/kaggle/working/CUET_run10_fixed.tsv", sep="\t", index=False, header=False)
# Vule run9 hishebe save kore felsi and run 10 hobe

print(f"\n✅ Regenerated and patched {len(patch_rows)} N/A questions.")


🔁 Regenerating for topic msmarco_v2.1_doc_21_861891150...

✅ Regenerated and patched 1 N/A questions.
